# Librerías

os: Para manejar directorios y archivos.

cv2: Librería OpenCV para procesamiento de imágenes.

numpy: Para cálculos matemáticos y operaciones con matrices.

In [ ]:
import os
import cv2
import numpy as np

# Funciones de Procesamiento de Imágenes

Implementé algunas funciones para tratar las imagenes, por lo que de una imagen se generan varias versiones, se guarda la imagen original y su versión con cada filtro.

La primera de ellas es para ajustar el brillo y contraste, consideré que sería bueno implementar esta función ya que podría ayudar a mejorar la generalización del modelo al exponerlo a imágenes con diferentes niveles de iluminación.

In [ ]:
def ajustar_brillo_contraste(frame, alpha=1.0, beta=0):
    return cv2.convertScaleAbs(frame, alpha=alpha, beta=beta)

Otra función es el recorte aleatorio, esto con la finalidad de extraer características específicas del coche, tomando en cuenta que en la imagenes podría salir solamente una parte del coche.

In [ ]:
def recortar_aleatoriamente(frame, porcentaje=0.1):
    h, w = frame.shape[:2]
    dy, dx = int(h * porcentaje), int(w * porcentaje)
    x1, y1 = np.random.randint(0, dx), np.random.randint(0, dy)
    return frame[y1:h - dy + y1, x1:w - dx + x1]

Esta función de zoom es para simular diferentes distancias 

In [ ]:
def aplicar_zoom(frame, factor=1.2):
    h, w = frame.shape[:2]
    nh, nw = int(h / factor), int(w / factor)
    frame_zoom = frame[(h - nh) // 2:(h + nh) // 2, (w - nw) // 2:(w + nw) // 2]
    return cv2.resize(frame_zoom, (w, h))

Esta versión de la imagen es para eliminar el ruido y que la imagen quede más clara.

In [ ]:
def eliminar_ruido(frame):
    return cv2.fastNlMeansDenoisingColored(frame, None, 10, 10, 7, 21)

Aquí se aplican los filtros blanco y negro, para ayudar en la parte de la detección de bordes y formas, rgb para ayudar a la clasificación de coches de acuerdo a los patrones de color y el filtro de escala de grises para el análisis de patrones y texturas.

In [ ]:
def aplicar_filtros_basicos(frame, filtro):
    if filtro == "blanco_negro":
        return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    elif filtro == "rgb":
        return cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    elif filtro == "escala_grises":
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        return cv2.merge([gray, gray, gray])
    else:
        return frame

Aquí se rota la imagen en un rango de ángulos aleatorios y se redimensiona, además se aumenta la variación en las posiciones de los coches en las imágenes.

In [ ]:
def rotar_y_redimensionar(frame, angulo_max=30, tamaño=(128, 128)):
    angulo = np.random.uniform(-angulo_max, angulo_max)
    
    h, w = frame.shape[:2]
    centro = (w // 2, h // 2)
    
    matriz_rotacion = cv2.getRotationMatrix2D(centro, angulo, 1)
    
    imagen_rotada = cv2.warpAffine(frame, matriz_rotacion, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))
    
    imagen_redimensionada = cv2.resize(imagen_rotada, tamaño)
    
    return imagen_redimensionada

Se aplica rotaciones predefinidas de 15, 30, 45, 60 y 90 grados, útil para garantizar variaciones consistentes en el dataset.

In [ ]:
def rotar_varias_veces(frame, angulos=[15, 30, 45, 60, 90], tamaño=(128, 128)):
    rotaciones = []
    for angulo in angulos:
        rotacion = rotar_y_redimensionar(frame, angulo_max=angulo, tamaño=tamaño)
        rotaciones.append(rotacion)
    return rotaciones

In [ ]:
def cambiar_tono_saturacion(frame, alpha=1.2, beta=50):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv[..., 0] = hsv[..., 0] * alpha + beta 
    hsv[..., 1] = hsv[..., 1] * alpha         
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

Aqupi se mandan a llamar las funciones de los filtros para crear las versiones de la imagen y guardarlas en formato jpg.

In [ ]:
def generar_versiones(output_folder, base_name, frame):
    transformaciones = {
        "rotacion_volteo": lambda img: cv2.flip(cv2.rotate(img, cv2.ROTATE_180), 1),
        "brillo_contraste": lambda img: ajustar_brillo_contraste(img, alpha=1.5, beta=30),
        "desplazamiento_zoom": lambda img: aplicar_zoom(img, factor=1.3),
        "recorte": lambda img: recortar_aleatoriamente(img, porcentaje=0.2),
        "eliminacion_ruido": lambda img: eliminar_ruido(img),
        "filtro_bn": lambda img: aplicar_filtros_basicos(img, "blanco_negro"),
        "filtro_rgb": lambda img: aplicar_filtros_basicos(img, "rgb"),
        "filtro_grises": lambda img: aplicar_filtros_basicos(img, "escala_grises"),
    }

    for nombre, transformacion in transformaciones.items():
        img_transformada = transformacion(frame)
        cv2.imwrite(f"{output_folder}/{base_name}_{nombre}.jpg", img_transformada)

Aqupi, creamos una carpeta para guardar el dataset en caso de que no exista

In [ ]:
def generar_dataset(video_url, output_folder, resolution=(28, 21), start_time=0, end_time=None):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

Luego, se descarga el vídeo, en este caso de youtube con la librería yt_dlp.

In [ ]:
ydl_opts = {
        'format': 'best',
        'outtmpl': 'temp_video.%(ext)s',
        'noplaylist': True,
        'quiet': True,
    }

with ytdlp.YoutubeDL(ydl_opts) as ydl:
    info_dict = ydl.extract_info(video_url, download=False)
    video_url = info_dict.get('url', None)
    print(f"Usando URL: {video_url}")

Se procesan los fotogramas

In [ ]:
cap = cv2.VideoCapture(video_url)

Se calculan los fps, la duración del vídeo y los frames, en este caso de inicio y fin ya que se define de que segundo a que segundo se toma del vídeo.

In [ ]:
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = total_frames / fps
print(f"Duración del video: {duration:.2f} segundos")
    
start_frame = int(start_time * fps)
end_frame = int(end_time * fps) if end_time is not None else total_frames

Se lee cada fotograma, se redimensiona, se guarda la imagen y genera las versiones

In [ ]:
cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
frame_count = start_frame
img_count = 0

cv2.namedWindow("Procesamiento", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Procesamiento", 600, 400)

while frame_count < end_frame:
    ret, frame = cap.read()
    if not ret:
        break

    cv2.imshow("Procesamiento", frame)

    if cv2.waitKey(1) & 0xFF == ord('x'):
        print("Ejecución detenida")
        break

    img_resized = cv2.resize(frame, resolution)

    base_name = f"frame_{img_count}"
    cv2.imwrite(f"{output_folder}/{base_name}.jpg", img_resized)

    generar_versiones(output_folder, base_name, img_resized)

    img_count += 1
    frame_count += 1
    print(f"Procesando frame {frame_count}/{total_frames}...")

cap.release()
cv2.destroyAllWindows()
print(f"Dataset generado en {output_folder}")

Aqupi se establece, primero el url del vídeo con el que vamos a trabajar, después, la carpeta donde se va a guardar el dataset y el tiempo inicial y final, en segundos, de la parte que tomaremos del vídeo.

In [ ]:
video_url = "https://youtu.be/pemqhq4qwDw?si=5k2_MW4CvE3bDWr0" 
output_folder = "C:/Users/ShiEu/Documents/dataset_coches"
start_time = 6
end_time = 12
generar_dataset(video_url, output_folder, start_time=start_time, end_time=end_time)
